In [37]:

import keras
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import flask
from keras.models import model_from_json

In [19]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [20]:
model=Model(inputs=base_model.input,outputs=preds)

In [21]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [22]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
#load data
train_generator=train_datagen.flow_from_directory('nailgun/nailgun',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 198 images belonging to 2 classes.


In [23]:
#train the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)
#save the model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Epoch 1/10
6/6 [==============================] - 117s 20s/step - loss: 2.3101 - acc: 0.5813
Epoch 2/10
6/6 [==============================] - 120s 20s/step - loss: 1.0510 - acc: 0.5479
Epoch 3/10
6/6 [==============================] - 107s 18s/step - loss: 0.4913 - acc: 0.8030
Epoch 4/10
6/6 [==============================] - 135s 22s/step - loss: 0.3493 - acc: 0.8438
Epoch 5/10
6/6 [==============================] - 91s 15s/step - loss: 0.3487 - acc: 0.8542
Epoch 6/10
6/6 [==============================] - 118s 20s/step - loss: 0.4450 - acc: 0.8282
Epoch 7/10
6/6 [==============================] - 136s 23s/step - loss: 0.3052 - acc: 0.8594
Epoch 8/10
6/6 [==============================] - 114s 19s/step - loss: 0.1646 - acc: 0.9681
Epoch 9/10
6/6 [==============================] - 129s 22s/step - loss: 0.0611 - acc: 0.9844
Epoch 10/10
6/6 [==============================] - 102s 17s/step - loss: 0.0468 - acc: 0.9945
Saved model to disk


NameError: name 'model_from_json' is not defined